In [ ]:

import pandas as pd
import os
from scipy.interpolate import interp1d
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from matplotlib import pyplot as plt
import networkx as nx

In [ ]:
path = r'datasets\\datasets_data\\collected\\'


In [32]:
#数据读取与预处理
xlsx = []
csv = []

#ignored: ecodata, air_condition

def read(path,date_key='时间',date_format = None,operations=None,dropsubset=None):
    sheet = pd.read_csv(path,index_col= date_key,parse_dates=[date_key],date_format=date_format)
    sheet.rename(columns={date_key:'date'},inplace=True)
    sheet.dropna(subset=dropsubset)
    if operations is not None:
        sheet = operations(sheet)
    sheet = sheet.resample('YE').mean()
    return sheet

# xlsx.append[([pd.read_excel(os.path.join(path,'carbon_emission.xlsx'))],'emission vector')]

# csv.append(read(os.path.join(path,'air_pollution_events.csv'),operations=lambda x: x.filter(regex='大气污染事故次数:年')))
csv.append(read(os.path.join(path,'citylize.csv')))
csv.append(read(os.path.join(path,'comsume.csv'),'时间?',operations=lambda x : x.filter(regex='支出')))
csv.append(read(os.path.join(path,'controllable_income.csv'),'时间'))
csv.append(read(os.path.join(path,'energy_investment.csv'),operations= lambda x: x.filter(regex='固定资产投资额(不含农户):能源工业:上海市:年')))
csv.append(read(os.path.join(path,'energy_structure.csv'),'时间?',operations=lambda x: x.filter(['能源生产总量构成:水电、核电、风电:年?','一次能源生产量:年?'])))
# csv.append(read(os.path.join(path,'medic_insurance.csv'),operations=lambda sheet: sheet.resample('YE').mean())) # 数据太少，影响整体质量，不使用
csv.append(read(os.path.join(path,'thick_cause_death.csv'),'时间?',operations= lambda x: x.filter(regex='(农村呼吸系统疾病粗死亡率_)|(城市呼吸系统疾病粗死亡率_)')))
csv.append(read(os.path.join(path,'work_time.csv'),operations=lambda sheet: sheet.resample('YE').mean()))
csv.append(read(os.path.join(path,'goverment_investment.csv'),'时间'))
csv.append(read(r'D:\huge_program_project\TJJM-Time-LLM\datasets\datasets_data\collected\processed\gdp\上海市_gdp_yearly_sum.csv',date_key='年份',operations=lambda x: x.rename(columns={'总量（亿元）':'gdp'})))
csv.append(read(r'D:\huge_program_project\TJJM-Time-LLM\datasets\datasets_data\collected\processed\co2\上海_emission.csv',date_key='year',operations=lambda x:x.rename(columns={'emission':'co2_emission'})))
csv.append(read(r'D:\huge_program_project\TJJM-Time-LLM\datasets\datasets_data\collected\processed\aqi\上海市_yearly_aircondition.csv',date_key='年份',operations=lambda x:x.rename(columns={'AQI':'当年平均空气质量指数'})))


csvs = pd.concat(csv,axis=1)
csvs = csvs.loc['2010-12-31':'2021-12-31']

rem = csvs.columns.difference(['date'])

for col in rem:
    csvs[col] = csvs[col].interpolate(method='spline', order=1)
    csvs[col] = csvs[col].dropna()
    

csvs.rename(columns={
    # '大气污染事故次数:年': '大气污染事故次数',
    '乡村人口数:年': '乡村人口数',
    '城镇人口数:年': '城镇人口数',
    '城乡居民比例': '城乡人口比',
    '农村家庭平均每人年消费性支出:年?': '农村人均消费支出',
    '城镇居民人均消费性支出:年?': '城镇人均消费支出',
    '居民可支配收入': '可支配收入',
    '能源生产总量构成:水电、核电、风电:年?': '清洁能源占比',
    '一次能源生产量:年?': '一次能源生产量', 
    '城市呼吸系统疾病粗死亡率_当期值_年?': '城市呼吸系统疾病死亡率',
    '农村呼吸系统疾病粗死亡率_当期值_年?': '农村呼吸系统疾病死亡率',
    '就业人员周平均工作时间:当期值:月': '周均工作时间',
    'gdp': 'GDP',
    'co2_emission': 'CO2排放量',
    '当年平均空气质量指数': '空气质量指数',
    '一般公共预算支出:年':'政府经费支出',
    '一般公共预算支出:节能环保支出:污染防治:年':'政府污染防治经费支出'
}, inplace=True)

#计算额外指标
csvs['环境治理支出占比'] = csvs['政府污染防治经费支出'] / csvs['政府经费支出'] * 100
# 保留两位小数
csvs['环境治理支出占比'] = csvs['环境治理支出占比'].round(2)


csvs.to_csv(os.path.join(path,'processed','summarize.csv'),index_label='date')

In [ ]:
#成本效益计算


In [34]:
#数据标准化
import joblib
# 读取summarize.csv
df = pd.read_csv(r'datasets\datasets_data\collected\processed\summarize.csv',index_col='date',parse_dates=['date'])
# 将'date'列转换为datetime类型
# df['date'] = pd.to_datetime(df['date'])

# 检查数据质量
print("数据形状:", df.shape)
# print("数据类型:\n", df.dtypes)
# print("缺失值检查:\n", df.isnull().sum())

# 假设整理后的DataFrame变量名为df
# 选择需要标准化的列
columns = [ '乡村人口数', '城镇人口数', '农村人均消费支出', 
                    '城镇人均消费支出', '可支配收入', '一次能源生产量',
                    '城市呼吸系统疾病死亡率', '农村呼吸系统疾病死亡率', '周均工作时间', 
                    'GDP', 'CO2排放量', '空气质量指数']

# 初始化StandardScaler
scaler = StandardScaler().fit(df[columns])

# 对选定的列进行标准化
df[columns] = scaler.transform(df[columns])

# print("预处理后的数据:\n", df.head())
df.to_csv(os.path.join(path,'processed',"data_preprocessed.csv"))  # 保存预处理后的数据
joblib.dump(scaler, os.path.join(path,'processed',"scalar.pkl")) #保存缩放因子方便后续训练

数据形状: (12, 17)


['datasets\\\\datasets_data\\\\collected\\\\processed\\scalar.pkl']

In [ ]:
#数据分析
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = ['SimHei']  # 使用中文字体

# 读取预处理后的数据
df = pd.read_csv(os.path.join(path,'processed',"data_preprocessed.csv"), index_col='date', parse_dates=['date'])

# Granger因果检验
maxlag = 2  # 设置最大时滞
test_results = pd.DataFrame(np.zeros((len(columns), len(columns))),
                            index=columns, columns=columns)
for cause in columns:
    for effect in columns:
        if cause != effect:
            test = grangercausalitytests(list(df[[cause, effect]].dropna().values), maxlag=maxlag, verbose=False)
            p_values = [test[i+1][0]['ssr_ftest'][1] for i in range(maxlag)]
            min_p_value = np.min(p_values)
            test_results.loc[cause, effect] = min_p_value

# 交叉相关分析
lags = range(1, 11)  # 设置时滞范围
cross_corr = pd.DataFrame(index=columns, columns=columns)
for cause in columns:
    for effect in columns:
        if cause != effect:
            cross_corr.loc[cause, effect] = df[cause].shift(1).corr(df[effect])


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 20))
sns.heatmap(test_results, ax=ax1, cmap='YlGnBu', annot=False,fmt='.3f')
ax1.set_title('Granger Causality')
sns.heatmap(cross_corr.astype(float), ax=ax2, cmap='YlOrRd', annot=False,fmt='.3f')
ax2.set_title('Cross Correlations')

plt.tight_layout()
plt.show()

In [42]:
#模型拟合
import pandas as pd
import numpy as np
import semopy
from semopy import Model
import os

# 读取数据
# data = pd.read_csv('data_preprocessed.csv', index_col='date', parse_dates=True)
data = pd.read_csv(os.path.join(path, 'processed', 'data_preprocessed.csv'), index_col='date', parse_dates=True)

model = """
# 潜变量
      社会经济地位 =~ 乡村人口数 + 城镇人口数 + 城乡人口比 + 农村人均消费支出 + 城镇人均消费支出 + 可支配收入 + GDP
      环境 =~ 清洁能源占比 + 一次能源生产量 + CO2排放量 + 空气质量指数
      健康 =~ 城市呼吸系统疾病死亡率 + 农村呼吸系统疾病死亡率
      工作状况 =~ 周均工作时间
      政策 =~ 政府经费支出 + 政府污染防治经费支出 + 环境治理支出占比
    
      # 观测变量
      乡村人口数 ~~ NA*乡村人口数
      城镇人口数 ~~ NA*城镇人口数  
      城乡人口比 ~~ NA*城乡人口比
      农村人均消费支出 ~~ NA*农村人均消费支出
      城镇人均消费支出 ~~ NA*城镇人均消费支出
      可支配收入 ~~ NA*可支配收入
      GDP ~~ NA*GDP
      清洁能源占比 ~~ NA*清洁能源占比
      一次能源生产量 ~~ NA*一次能源生产量
      CO2排放量 ~~ NA*CO2排放量
      空气质量指数 ~~ NA*空气质量指数
      城市呼吸系统疾病死亡率 ~~ NA*城市呼吸系统疾病死亡率
      农村呼吸系统疾病死亡率 ~~ NA*农村呼吸系统疾病死亡率
      周均工作时间 ~~ NA*周均工作时间
      政府经费支出 ~~ NA*政府经费支出
      政府污染防治经费支出 ~~ NA*政府污染防治经费支出
      环境治理支出占比 ~~ NA*环境治理支出占比
    
      # 路径关系  
      健康 ~ 环境 + 社会经济地位 + 工作状况
      环境 ~ 社会经济地位 + 政策
      工作状况 ~ 社会经济地位
      社会经济地位 ~ 政策
"""

# 定义模型
model = Model(model)

# 拟合模型
print(f'fit result:\n {model.fit(data)}')

# 输出结果
# print(model.inspect())

print(semopy.calc_stats(model).T)
g = semopy.semplot(model, os.path.join("article","figures","sem_model.png"),show=True)

# # 计算成本效益
# def cost_benefit(policy_cost, health_benefit, discount_rate=0.05, time_horizon=20):
#     net_benefit = 0
#     for t in range(time_horizon):
#         net_benefit += (health_benefit - policy_cost) / (1 + discount_rate)**t
#     return net_benefit

# policy_cost = 1000  # 假设的政策成本
# health_benefit = model.predict(data, "Health") - model.predict(data, "Health")
# health_benefit = health_benefit.mean() * 10000  # 假设每个单位对应1万元的健康效益

# net_benefit = cost_benefit(policy_cost, health_benefit)
# print(f"净效益: {net_benefit}")


fit result:
 Name of objective: MLW
Optimization method: SLSQP
Optimization successful.
Optimization terminated successfully
Objective value: 602.289
Number of iterations: 2
Params: -0.999 -0.047 -0.994 -0.994 0.831 -0.694 0.227 -0.149 -0.229 0.776 10816.537 1.934 8812020.223 0.055 0.051 8813152.981 0.103 0.089 -0.002 0.001 -0.000 -0.005 67.750 -0.001 -62.283
                     Value
DoF             128.000000
DoF Baseline    152.000000
chi2           7227.472296
chi2 p-value      0.000000
chi2 Baseline  6398.866016
CFI              -0.136485
GFI              -0.129493
AGFI             -0.341273
NFI              -0.129493
TLI              -0.349576
RMSEA             2.245493
AIC           -1154.578716
BIC           -1142.456050
LogLik          602.289358
